In [99]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
from mprompt.modules.emb_diff_module import EmbDiffModule
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
import sasc.notebook_helper
import scipy.special
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from mprompt.methods.m4_evaluate import D5_Validator
import openai
from mprompt.modules.fmri_module import fMRIModule
from pprint import pprint
import joblib
from mprompt.config import RESULTS_DIR
import torch.cuda

r = (pd.read_pickle('../results/results_fmri_full.pkl')
    .sort_values(by=['top_score_synthetic'], ascending=False))

EXPT_NAME = 'relationships_mar9'
rows = joblib.load(join(RESULTS_DIR, 'stories', f'{EXPT_NAME}_rows.pkl'))
expls = rows.expl.values
voxel_nums = rows.module_num.values
subjects = rows.subject.values
paragraphs = rows.paragraph.values
prompts = rows.prompt.values

scores_all_data = joblib.load(join(RESULTS_DIR, 'stories', f'{EXPT_NAME}_scores_data.pkl'))['scores_all']
# keys: scores_mod, scores_max_mod, all_scores, all_ngrams
scores_all_mod = joblib.load(join(RESULTS_DIR, 'stories', f'{EXPT_NAME}_scores_mod.pkl'))['scores_all']
# scores_mod = scores_mod_dict['scores_mod']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Examples

In [100]:
rows
columns = {
    # explanation
    'top_explanation_init_strs': 'Explanation',

    # fmri stuff
    'subject': 'Subject',
    # 'module_num': 'Voxel',
    'roi_func': 'ROI',
    # 'roi_anat': 'ROI (anatomical)',
    'fmri_test_corr': '\\makecell[r]{fMRI correlation\\\\(test)}',
    
    # scores
    'top_score_synthetic': '\\makecell[r]{Synthetic\\\\data score}',
    'frac_top_ngrams_module_correct': '\\makecell[r]{Fraction of matching\\\\ngrams (module, top-75)}',
    # 'rankcorr_expl_test': 'Correlation (test) when predicting with only explanation',
    # 'frac_top_ngrams_test_correct': 'Fraction of matching ngrams (test, top-75)',
        
    # ngrams matching the explanation (used 75 ngrams)
    # 'top_ngrams_module_correct': 'Matching top ngrams (out of top-75)',
    # 'top_ngrams_test_correct': 'Matching ngrams (test, top-75)',

    # all ngrams
    # 'top_ngrams_module_25': 'All top ngrams (top-25)',
    # 'top_ngrams_test_25': 'Top ngrams (test, top-25)',

    # alternative explanations
    # 'explanation_init_strs': 'Explanation candidates',
}

tab = (
    rows
    .sort_values(by=['top_score_synthetic'], ascending=False)
    # .sort_values(by=['rankcorr_expl_test'], ascending=False)
    .filter(columns.keys())
    .rename(columns=columns)
    .round(3)
)
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    tab_join_lists = tab.applymap(lambda x: ', '.join(x) if isinstance(x, np.ndarray) or isinstance(x, list) else x)
    # display(
        # tab_join_lists.head(3)
    # )
    print(tab_join_lists.style.hide(axis='index').format(precision=2).to_latex(hrules=True))
    # tab_join_lists.to_csv('../results/results_fmri.csv', index=False, float_format='%.3f')

\begin{tabular}{lllrrr}
\toprule
Explanation & Subject & ROI & \makecell[r]{fMRI correlation\\(test)} & \makecell[r]{Synthetic\\data score} & \makecell[r]{Fraction of matching\\ngrams (module, top-75)} \\
\midrule
looking or staring in some way & UTS03 & IFSFP & 0.53 & 0.10 & 0.40 \\
relationships and milestones in life & UTS02 & PMvh, AC & 0.35 & 0.07 & 0.55 \\
passing of time & UTS02 & PMvh, AC & 0.41 & 0.07 & 0.28 \\
food and drink & UTS01 & Broca & 0.41 & 0.06 & 0.52 \\
hands and arms & UTS01 & EBA, PMvh, S2F & 0.30 & 0.05 & 0.87 \\
\bottomrule
\end{tabular}



### Table metrics

In [117]:
def get_diff_and_sem_diag_vs_off_diag(arr: np.ndarray[List]):
    # difference between diagonal and off-diagonal
    eye = np.eye(arr.shape[0]).astype(bool)
    diag = arr[eye]
    off_diag = arr[~eye]

    # individual elements
    diags = np.hstack(diag).astype(float).tolist()
    off_diags = np.hstack(off_diag).astype(float).tolist()
    diff = np.mean(diags) - np.mean(off_diags)
    # print('meandiags', np.mean(diags), 'meanoff', np.mean(off_diags))
    sem = np.sqrt(np.var(diags) / len(diags) + np.var(off_diags) / len(off_diags))
    return diff, sem

mets = {}
diff, sem = get_diff_and_sem_diag_vs_off_diag(np.array(scores_all_data))
mets['Explanation difference'] = [f'{diff:.3f} \err' + '{' + f'{sem:.3f}' + '}']
diff, sem = get_diff_and_sem_diag_vs_off_diag(np.array(scores_all_mod))
mets['Module response difference'] = [f'{diff:.3f} \err' + '{' + f'{sem:.3f}' + '}']
n = r.shape[0]
mets['Test rank correlation'] = [f'{r["rankcorr_expl_test"].mean():.3f} \err' + '{' + f'{r["rankcorr_expl_test"].std()/np.sqrt(n):.3f}' + '}']

meandiags 0.2547008547008547 meanoff 0.041025641025641026
meandiags 0.044020334554224334 meanoff 0.018610744476272468


/tmp/ipykernel_2099606/2840335649.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  diff, sem = get_diff_and_sem_diag_vs_off_diag(np.array(scores_all_data))
/tmp/ipykernel_2099606/2840335649.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  diff, sem = get_diff_and_sem_diag_vs_off_diag(np.array(scores_all_mod))


In [114]:
print(pd.DataFrame.from_dict(mets).style.hide(axis='index').format(precision=2).to_latex(hrules=True))

\begin{tabular}{lll}
\toprule
Explanation difference & Module response difference & Test rank correlation \\
\midrule
0.214 \err{0.018} & 0.025 \err{0.005} & 0.031 \err{0.005} \\
\bottomrule
\end{tabular}

